In [20]:
from web3 import Web3
from tqdm import tqdm
from joblib import Parallel, delayed

In [3]:
# Use Moralis endpoint
endpoint = "https://speedy-nodes-nyc.moralis.io/03c966587b022c980f59136b/eth/mainnet/archive"
w3 = Web3(Web3.HTTPProvider(endpoint))
w3.isConnected()

True

# Step 1: Get transaction hashes of Audioglyphs NFT transfers
https://www.quicknode.com/docs/ethereum/eth_getLogs

Moralis restricts us to a maximum block range of 2000, so we have to iterate

In [16]:
w3.eth.get_logs({"fromBlock": str(hex(14547199)),
                 "toBlock": str(hex(14547199+2000)),
                 "address": Web3.toChecksumAddress("0xfb3765e0e7ac73e736566af913fa58c3cfd686b7"),
                 "topics": ["0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef"]})

[AttributeDict({'address': '0xFb3765E0E7AC73E736566AF913FA58c3cFD686b7',
  'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'),
   HexBytes('0x000000000000000000000000fb865683277e54658dab0fa90e049616cb7f254c'),
   HexBytes('0x000000000000000000000000fb174c7222f5737a86a58d96d269708f21d340cd'),
   HexBytes('0x00000000000000000000000000000000000000000000000000000000000016cb')],
  'data': '0x',
  'blockNumber': 14547200,
  'transactionHash': HexBytes('0xecd05a0552b46d3fcb87b8c79422b0fc7a53b7a6d93a91cbaa8fb2234e162f18'),
  'transactionIndex': 65,
  'blockHash': HexBytes('0x0315c4ac268896872afe67d36ee0e935d4849225162f3b3a7283dac0f433cdd0'),
  'logIndex': 105,
  'removed': False})]

In [116]:
frstBlock = 14200000
lastBlock = 14210000
blockRange = 2000 # Moralis only supports a maximum of 2000

txHashList = []
for i in tqdm(range(frstBlock, lastBlock, blockRange)):
    response = w3.eth.get_logs({"fromBlock": str(hex(i)),
                                "toBlock": str(hex(i+blockRange)),
                                "address": Web3.toChecksumAddress("0xfb3765e0e7ac73e736566af913fa58c3cfd686b7"),
                                "topics": ["0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef"]})
    for item in response:
        txHashList.append(item.transactionHash.hex())
txHashList = list(set(txHashList)) # remove duplicates
print("Got",len(txHashList), "transactions with transfers")

100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [00:03<00:00,  1.57it/s]

Got 39 transactions with transfers


# Step 2: Get all events related to these transaction hashes

In [54]:
#from web3._utils.events import get_event_data

In [142]:
from pathlib import Path
from web3.logs import STRICT, IGNORE, DISCARD, WARN
transferLogs = []

myContract = w3.eth.contract(address="0xFb3765E0E7AC73E736566AF913FA58c3cFD686b7",
                             abi=Path('AudioGlyphs.json').read_text())

for txHash in txHashList:
    receipt = w3.eth.getTransactionReceipt(txHashList[2])
    logs = myContract.events.Transfer().processReceipt(receipt, errors=DISCARD)
    for log in logs:
        log = dict(log)
        log["transactionHash"] = log["transactionHash"].hex()
        del log["blockHash"]
        for k,v in log["args"].items():
            log[k] = v
        del log["args"]
        transferLogs.append(log)

In [141]:
pd.DataFrame(transferLogs)

,event,logIndex,transactionIndex,transactionHash,address,blockHash,blockNumber,from,to,tokenId
0,Transfer,261,410,b'\x8b\x1e\xdc\xdaI\xe1\x15\x86\xa7\xa3>\xa8\\...,0xD6A592367e0DEAE0059B66babfA069E446660acB,"b'},\xe8l\x93\xd8\xb7L\x9a\x19{\xfb\x95\x8e\xc...",14204085,0x00C71F8C497D8950553Fcb874F4A8Cf74Dc88629,0x2AAcC0A4fEc535aCfAdd2feb09D8Db272b7458d2,73
1,Transfer,263,410,b'\x8b\x1e\xdc\xdaI\xe1\x15\x86\xa7\xa3>\xa8\\...,0x49aC61f2202f6A2f108D59E77535337Ea41F6540,"b'},\xe8l\x93\xd8\xb7L\x9a\x19{\xfb\x95\x8e\xc...",14204085,0x00C71F8C497D8950553Fcb874F4A8Cf74Dc88629,0x2AAcC0A4fEc535aCfAdd2feb09D8Db272b7458d2,164
2,Transfer,265,410,b'\x8b\x1e\xdc\xdaI\xe1\x15\x86\xa7\xa3>\xa8\\...,0x49aC61f2202f6A2f108D59E77535337Ea41F6540,"b'},\xe8l\x93\xd8\xb7L\x9a\x19{\xfb\x95\x8e\xc...",14204085,0x00C71F8C497D8950553Fcb874F4A8Cf74Dc88629,0x2AAcC0A4fEc535aCfAdd2feb09D8Db272b7458d2,9776
3,Transfer,267,410,b'\x8b\x1e\xdc\xdaI\xe1\x15\x86\xa7\xa3>\xa8\\...,0xFb3765E0E7AC73E736566AF913FA58c3cFD686b7,"b'},\xe8l\x93\xd8\xb7L\x9a\x19{\xfb\x95\x8e\xc...",14204085,0x00C71F8C497D8950553Fcb874F4A8Cf74Dc88629,0x2AAcC0A4fEc535aCfAdd2feb09D8Db272b7458d2,6410
4,Transfer,261,410,b'\x8b\x1e\xdc\xdaI\xe1\x15\x86\xa7\xa3>\xa8\\...,0xD6A592367e0DEAE0059B66babfA069E446660acB,"b'},\xe8l\x93\xd8\xb7L\x9a\x19{\xfb\x95\x8e\xc...",14204085,0x00C71F8C497D8950553Fcb874F4A8Cf74Dc88629,0x2AAcC0A4fEc535aCfAdd2feb09D8Db272b7458d2,73
...,...,...,...,...,...,...,...,...,...,...
151,Transfer,267,410,b'\x8b\x1e\xdc\xdaI\xe1\x15\x86\xa7\xa3>\xa8\\...,0xFb3765E0E7AC73E736566AF913FA58c3cFD686b7,"b'},\xe8l\x93\xd8\xb7L\x9a\x19{\xfb\x95\x8e\xc...",14204085,0x00C71F8C497D8950553Fcb874F4A8Cf74Dc88629,0x2AAcC0A4fEc535aCfAdd2feb09D8Db272b7458d2,6410
152,Transfer,261,410,b'\x8b\x1e\xdc\xdaI\xe1\x15\x86\xa7\xa3>\xa8\\...,0xD6A592367e0DEAE0059B66babfA069E446660acB,"b'},\xe8l\x93\xd8\xb7L\x9a\x19{\xfb\x95\x8e\xc...",14204085,0x00C71F8C497D8950553Fcb874F4A8Cf74Dc88629,0x2AAcC0A4fEc535aCfAdd2feb09D8Db272b7458d2,73
153,Transfer,263,410,b'\x8b\x1e\xdc\xdaI\xe1\x15\x86\xa7\xa3>\xa8\\...,0x49aC61f2202f6A2f108D59E77535337Ea41F6540,"b'},\xe8l\x93\xd8\xb7L\x9a\x19{\xfb\x95\x8e\xc...",14204085,0x00C71F8C497D8950553Fcb874F4A8Cf74Dc88629,0x2AAcC0A4fEc535aCfAdd2feb09D8Db272b7458d2,164
154,Transfer,265,410,b'\x8b\x1e\xdc\xdaI\xe1\x15\x86\xa7\xa3>\xa8\\...,0x49aC61f2202f6A2f108D59E77535337Ea41F6540,"b'},\xe8l\x93\xd8\xb7L\x9a\x19{\xfb\x95\x8e\xc...",14204085,0x00C71F8C497D8950553Fcb874F4A8Cf74Dc88629,0x2AAcC0A4fEc535aCfAdd2feb09D8Db272b7458d2,9776


In [93]:
ordersMatchedTopic = "0xc4109843e0b7d514e4c093114b863f8e7d8d9a458c372cd51bfe526b588006c9"
ordersMatchedEvents = []
transferTopic = "0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef"
transferEvents = []
for txHash in txHashList:
    receipt = w3.eth.get_transaction_receipt(txHash)
    logs = receipt.logs
    for log in logs:
        topics = log.topics
        eventType = topics[0].hex()
        if eventType == ordersMatchedTopic:
            ordersMatchedEvents.append(log)
        elif (eventType == transferTopic) & (len(topics) == 4):
            transferEvents.append({"from":topics[1].hex(), "to":topics[2].hex(), "NFTid":topics[3].hex()})

In [95]:
pd.DataFrame(transferEvents)

,from,to,NFTid
0,0x00000000000000000000000099c052567b617345398c...,0x0000000000000000000000009b0af8e61bbb1d8c6f43...,0x00000000000000000000000000000000000000000000...
1,0x000000000000000000000000a7d30065391e7c2fdbbe...,0x000000000000000000000000a2a8894c5533133f7c67...,0x00000000000000000000000000000000000000000000...
2,0x00000000000000000000000000c71f8c497d8950553f...,0x0000000000000000000000002aacc0a4fec535acfadd...,0x00000000000000000000000000000000000000000000...
3,0x00000000000000000000000000c71f8c497d8950553f...,0x0000000000000000000000002aacc0a4fec535acfadd...,0x00000000000000000000000000000000000000000000...
4,0x00000000000000000000000000c71f8c497d8950553f...,0x0000000000000000000000002aacc0a4fec535acfadd...,0x00000000000000000000000000000000000000000000...
5,0x00000000000000000000000000c71f8c497d8950553f...,0x0000000000000000000000002aacc0a4fec535acfadd...,0x00000000000000000000000000000000000000000000...
6,0x00000000000000000000000055282522c503a4f98d8b...,0x000000000000000000000000c1281c3dbd892a96c9ea...,0x00000000000000000000000000000000000000000000...
7,0x000000000000000000000000a2a8894c5533133f7c67...,0x000000000000000000000000a7d30065391e7c2fdbbe...,0x00000000000000000000000000000000000000000000...
8,0x00000000000000000000000099c052567b617345398c...,0x0000000000000000000000009b0af8e61bbb1d8c6f43...,0x00000000000000000000000000000000000000000000...
9,0x0000000000000000000000009b0af8e61bbb1d8c6f43...,0x00000000000000000000000099c052567b617345398c...,0x00000000000000000000000000000000000000000000...


In [81]:
import pandas as pd
data = pd.DataFrame(allLogs)
data.topics

0      [b'\x8c[\xe1\xe5\xeb\xec}[\xd1OqB}\x1e\x84\xf3...
1      [b'\xdd\xf2R\xad\x1b\xe2\xc8\x9bi\xc2\xb0h\xfc...
2      [b'\xc4\x10\x98C\xe0\xb7\xd5\x14\xe4\xc0\x93\x...
3      [b'\x8c[\xe1\xe5\xeb\xec}[\xd1OqB}\x1e\x84\xf3...
4      [b'\xdd\xf2R\xad\x1b\xe2\xc8\x9bi\xc2\xb0h\xfc...
                             ...                        
133    [b'\x8c[\xe1\xe5\xeb\xec}[\xd1OqB}\x1e\x84\xf3...
134    [b'\xdd\xf2R\xad\x1b\xe2\xc8\x9bi\xc2\xb0h\xfc...
135    [b'\x8c[\xe1\xe5\xeb\xec}[\xd1OqB}\x1e\x84\xf3...
136    [b'\xdd\xf2R\xad\x1b\xe2\xc8\x9bi\xc2\xb0h\xfc...
137    [b'\xc4\x10\x98C\xe0\xb7\xd5\x14\xe4\xc0\x93\x...
Name: topics, Length: 138, dtype: object

In [84]:
allLogs[0].topics[0].hex()

'0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925'

In [51]:
response = w3.eth.get_logs({"fromBlock": str(hex(frstBlock)),
                                "toBlock": str(hex(frstBlock+blockRange)),
                                "address": Web3.toChecksumAddress("0xfb3765e0e7ac73e736566af913fa58c3cfd686b7"),
                                "topics": ["0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef"]})

AttributeDict({'blockHash': HexBytes('0x0315c4ac268896872afe67d36ee0e935d4849225162f3b3a7283dac0f433cdd0'),
 'blockNumber': 14547200,
 'contractAddress': None,
 'cumulativeGasUsed': 4960517,
 'effectiveGasPrice': 50264459892,
 'from': '0xFB174C7222F5737a86A58d96d269708f21d340Cd',
 'gasUsed': 241875,
 'logs': [AttributeDict({'address': '0xFb3765E0E7AC73E736566AF913FA58c3cFD686b7',
   'topics': [HexBytes('0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925'),
    HexBytes('0x000000000000000000000000fb865683277e54658dab0fa90e049616cb7f254c'),
    HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
    HexBytes('0x00000000000000000000000000000000000000000000000000000000000016cb')],
   'data': '0x',
   'blockNumber': 14547200,
   'transactionHash': HexBytes('0xecd05a0552b46d3fcb87b8c79422b0fc7a53b7a6d93a91cbaa8fb2234e162f18'),
   'transactionIndex': 65,
   'blockHash': HexBytes('0x0315c4ac268896872afe67d36ee0e935d4849225162f3b3a7283dac0f433cdd0'),
